<a href="https://colab.research.google.com/github/stupidbutsmart/DELE-CA1/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Section A CA1 CNN

Name: Jovan Heng Ghim Hong

Class: DAAA/2B/22

Admin No: 2401418

In [5]:
# Basic Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten , Conv2D
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.preprocessing import image

## for keras classifier i will be using scikeras lib
## 2.12+ no longer supports tensorflow.keras.wrappers.scikit_learn
## this was coded in colab, thus 2.19 is used
from scikeras.wrappers import KerasClassifier


warnings.filterwarnings('ignore');

### Creating UDFs to aid workflow

Before i start, i want to create some functions that can help me with my work.

basic functions like:
1. create model function
2. autoencoder function
3. data processing functions

etc.

This can help modularize my code (especially since im working with 2 models) and make it easier for future collaborater to understand

In [37]:
DATASET_PATH = './datasets/'

def get_data(data_type , size):
  targets = [ target for target in os.listdir(DATASET_PATH + data_type)]

  rows = []
  for target in targets: 
    for file_name in os.listdir(os.path.join(DATASET_PATH , data_type ,  target)):
      img = image.load_img(os.path.join(DATASET_PATH , data_type, target , file_name ) , target_size=size)
      img = image.img_to_array(img)
      
      record = {'sequence' : img}
      for t in targets:
        record[t] = 1 if t == target else 0

      rows.append(record)

  return pd.DataFrame(rows , columns=['sequence'] + targets)



In [38]:
def create_model(
    number_of_layers=3 , number_of_neurons=128,
    optimizer='adam' , activation='relu',
    layers=[Conv2D], # incase i want to add more layers
    strict_follow_layers = False,
    input_shape=(23 , 23 , 3) # follows first layer
):
  model = Sequential()

  # Adding layers
  if strict_follow_layers:
    for layer in layers:
      model.add(layer(number_of_neurons , activation=activation))
  else:
    for i in range(number_of_layers):
      for layer in layers:
        model.add(layer(number_of_neurons , activation=activation))

  model.add(Flatten())
  model.add(Dense(1 , activation='softmax'))
  model.compile(optimizer=optimizer , loss='categorical_crossentropy' , metrics=['accuracy'])

  return KerasClassifier(model)



In [39]:
train_data = get_data('train' , (23 , 23))

In [41]:
train_data.head()

,sequence,Bean,Bitter_Gourd,Brinjal,Cabbage,Capsicum,Cauliflower and Broccoli,Cucumber and Bottle_Gourd,Potato,Pumpkin,Radish and Carrot,Tomato
0,"[[[21.0, 38.0, 46.0], [90.0, 107.0, 150.0], [2...",1,0,0,0,0,0,0,0,0,0,0
1,"[[[185.0, 189.0, 198.0], [184.0, 188.0, 197.0]...",1,0,0,0,0,0,0,0,0,0,0
2,"[[[173.0, 95.0, 59.0], [160.0, 90.0, 54.0], [1...",1,0,0,0,0,0,0,0,0,0,0
3,"[[[52.0, 53.0, 57.0], [54.0, 55.0, 59.0], [41....",1,0,0,0,0,0,0,0,0,0,0
4,"[[[161.0, 162.0, 167.0], [161.0, 166.0, 170.0]...",1,0,0,0,0,0,0,0,0,0,0
